# EEG - Flow

## 4. ICA select

Last edit: 17.05.2023 16:08

In [ ]:
from mne import set_log_level as set_log_level_mne
from mne.viz import set_browser_backend

from eeg_flow import set_log_level
from eeg_flow.tasks import label_components

In [ ]:
set_log_level_mne("WARNING")
set_log_level("INFO")

%matplotlib qt
set_browser_backend('qt')

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
PARTICIPANT: str = ""  # e.g. "P02"
GROUP: str = ""  # e.g. "G2"
TASK: str = ""  # "oddball" or "UT"
RUN: int = 1  # 1 or 2

This step is divided in annotation of the first ICA (for mastoids) and annotation of the second ICA (for EEG).

The first ICA's goal is to remove noise present on the mastoids (reference). Any artifact topographies that do not impact the mastoids are disregarded. Typically, heartbeat or muscle noise can affect the mastoids.

The second ICA is fitted on data without the mastoids. The goal is now to remove noise on the EEG channels.

The created derivatives are:
- ICA decomposition labelled used to clean-up mastoids (`_step4_1st_{reviewer}_ica.fif`)
- ICA decomposition labelled used to clean-up signal (`_step4_2nd_{reviewer}_ica.fif`)
- Plots of the ICA topographies (in `plots/ica`)

### Interactive selection

In both cases, the selection should be done on the 2D data browser by marking as "bads" the traces (time-series) of each ICs to reject. The selection by selecting the name of the ICs to reject is disabled on the topographies.

In [ ]:
label_components(PARTICIPANT, GROUP, TASK, RUN)